# Lab03: Web Crawler (Continue) & Information Retrieval.

- MSSV: 18600187
- Họ và tên: Vũ Cao Nguyên

## Yêu cầu bài tập

**Cách làm bài**


Bạn sẽ làm trực tiếp trên file notebook này; từ `TODO` cho biết những phần mà bạn cần phải làm.

Bạn có thể thảo luận ý tưởng cũng như tham khảo các tài liệu, nhưng *code và bài làm phải là của bạn*. 

Nếu vi phạm thì sẽ bị 0 điểm cho bài tập này.

**Cách nộp bài**

Trước khi nộp bài, rerun lại notebook (`Kernel` -> `Restart & Run All`).

Sau đó, tạo thư mục có tên `MSSV` của bạn (vd, nếu bạn có MSSV là 1234567 thì bạn đặt tên thư mục là `1234567`) Chép file notebook, file `t_data.txt` và file `raw_data` của các bạn (nếu file này kích thước lớn các bạn có thể chép link vào `link_data.txt`), nén thư mục `MSSV` này lại và nộp trên moodle.

**Nội dung bài tập**

Cài đặt một web crawler để thu thập dữ liệu từ: https://en.wikipedia.org/wiki/Web_mining.

## Nội dung bài tập

Cài đặt một Web crawler đơn giản bắt đầu từ URL: https://en.wikipedia.org/wiki/Web_mining, tìm liên kết và thu thập dữ liệu trong HTML tại URL này sau đó lặp lại với các URL vừa tìm được.


## 1. Thu thập đường dẫn

- Robot.txt:  https://en.wikipedia.org/robots.txt
- **Bước 1**: Thu thập đường dẫn từ https://en.wikipedia.org/wiki/Web_mining. Lưu trữ vào một danh sách `url_list`. 
- **Bước 2**: Lặp lại bước 1 cho các đường dẫn trong list_url (**lưu ý:** kiểm tra các đường dẫn vừa thu được đã nằm trong `url_list` hay không?). Dừng khi đã thu thập được 200 URLs.

In [ ]:
import requests
import re
from bs4 import BeautifulSoup
from bs4.element import Comment
import string
import pickle
import urllib.robotparser

rp = urllib.robotparser.RobotFileParser()
rp.set_url("https://en.wikipedia.org/robots.txt")
rp.read()
rrate = rp.request_rate("*")

rp.crawl_delay("*")

def get_urls(url):
    r = requests.get(url)
    urls = re.findall('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', r.text) 
    urls_copy = urls.copy()
    for url in urls_copy:
      if(rp.can_fetch("*", url) == False):
        urls.remove(url)
    return urls

def get_urls_recursive(start_url, limit):
    urls = get_urls(start_url)
    for url in urls:
        if(len(urls) >= limit):
            return urls[:limit]
        new_urls = get_urls(url)
        for new_url in new_urls:
          if(new_url not in urls and len(urls) < limit):
            urls.append(new_url)
    return urls[:limit]
url_list = get_urls_recursive('https://en.wikipedia.org/wiki/Web_mining', 200)
print((url_list))
print(len(url_list))

['https://en.wikipedia.org/wiki/Web_mining', 'https://archive.org/details/webinformationsy00ngua', 'https://archive.org/details/webinformationsy00ngua/page/n33', 'https://archive.org/details/electroniccommer00bauk_698', 'https://archive.org/details/electroniccommer00bauk_698/page/n176', 'https://archive.org/details/webminingapplica0000scim/page/282', 'https://archive.org/details/webminingapplica0000scim/page/282', 'http://alexandria.tue.nl/repository/freearticles/612259.pdf', 'http://www.cis.unisa.edu.au/', 'https://docs.google.com/open?id=1nU1vrz-gBtSJk3bkb1ls_QuGX2nUPPemECPFlCx0C75MvmQdSqPci6LZWJYf', 'https://ui.adsabs.harvard.edu/abs/2000cs.......11033K', 'https://doi.org/10.1145%2F360402.360406', 'https://api.semanticscholar.org/CorpusID:60455', 'http://library.ifla.org/148/', 'https://doi.org/10.5195%2Fjmla.2019.650', 'http://soave.isti.cnr.it/%7Esilvestr/wp-content/uploads/2007/03/p63-baraglia.pdf', 'http://facweb.cs.depaul.edu/mobasher/classes/ect584/papers/cms-kais.pdf', 'https

## 2. Thu thập dữ liệu
Thu thập dữ liệu từ `url_list`. Lưu trữ dữ liệu thu được vào dictionary data với keys là các từ, values gồm 2 phần tử: 
- `url_idx_list` với $idx \in \left[0,200\right) \cap \mathbb{N}$
- `frequency` 
    
Ví dụ: `data['at]=[url_idx_list,frequency]`:
- `url_idx_list`: danh sách các url mà trong dữ liệu của chúng (html document) chứa từ "at". 
- `frequency`: tần suất xuất hiện (số lần xuất hiện) của từ `at` trong dữ liệu của **tất cả đường dẫn thu được**.   

In [ ]:
def text_filter(element):
    if element.parent.name in ['style', 'title', 'script', 'head', '[document]', 'class', 'a', 'li']:
        return False
    elif isinstance(element, Comment):
        '''Opinion mining?'''
        return False
    elif re.match(r"[\s\r\n]+",str(element)): 
        '''space, return, endline'''
        return False
    return True

def wordList(url):
    r=requests.get(url)
    soup=BeautifulSoup(r.content,"html.parser")
    text=soup.findAll(text=True)
    filtered_text=list(filter(text_filter,text))
    word_list=[]
    #TODO:
    for astring in filtered_text:
      for types in string.punctuation:
        astring=astring.replace(types,' ')
        word_list.extend(astring.split(' '))
        word_list=[astr for astr in word_list if astr not in ' \n']
    return word_list

def read_url(url, url_idx, data):
    word_list=wordList(url)
    
    #TODO
    '''Initialize value: data[w] = [[url_idx], 1]'''
                                    #list_url
    for w in word_list:
      w = w.lower()
      if data.get(w) is None:
        data[w] = [[url_idx],1]
      else:
        if url_idx not in data[w][0]:
          data[w][0].append(url_idx)
        else:
          data[w][1] +=1
    return 1  

In [ ]:
data = {}
for url_index, url in enumerate(url_list, 1):
    read_url(url, url_index, data)
print(data)
print(len(data))

{'web': [[1, 2, 3, 4, 5], 2524], 'mining': [[1, 2, 3], 1411], 'from': [[1, 2, 3], 350], 'wikipedia,': [[1], 11], 'the': [[1, 2, 3, 4, 5], 2300], 'free': [[1], 32], 'encyclopedia': [[1], 32], 'wikipedia': [[1], 21], 'this': [[1, 2, 3, 4, 5], 924], 'article': [[1], 96], 'includes': [[1, 2, 3, 4, 5], 156], 'a': [[1, 2, 3, 4, 5], 1781], ',': [[1, 2, 3, 4, 5], 546], 'related': [[1, 2, 3, 4, 5], 140], 'reading': [[1], 32], 'or': [[1, 2, 3, 4, 5], 316], 'but': [[1], 128], 'its': [[1], 32], 'sources': [[1], 51], 'remain': [[1], 32], 'unclear': [[1], 32], 'because': [[1], 32], 'it': [[1], 192], 'lacks': [[1], 32], '.': [[1], 507], '(': [[1], 28], 'october': [[1, 4, 5], 158], '2020': [[1], 96], ')': [[1], 56], 'may': [[1], 32], 'require': [[1], 32], 'specific': [[1], 128], 'problem': [[1], 32], 'is:': [[1], 15], 'is': [[1, 2, 3], 559], 'needs': [[1], 96], 'sufficient': [[1], 32], 'inline': [[1], 32], 'references,': [[1], 11], 'and': [[1, 2, 3, 4, 5], 2012], 'complete': [[1], 32], 're-write': [[1

## 3. Tiền xử lý
Loại bỏ các item trong data mà key là các stopword.

**Ngữ liệu:** 

In [ ]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
english_stopwords = stopwords.words('english')

data_copy = data.copy()
for key in data_copy:
  
  if key in english_stopwords:
    print(key)
    data.pop(key)
print(data)
print(len(data))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
from
the
this
a
or
but
its
because
it
is
and
has
been
re
as
by
of
to
both
about
can
have
be
for
will
in
itself
into
between
with
their
at
further
on
which
are
more
such
does
not
when
an
if
s
then
that
being
these
any
they
there
no
them
some
against
you
do
nor
we
under
other
over
up
were
off
{'web': [[1, 2, 3, 4, 5], 2524], 'mining': [[1, 2, 3], 1411], 'wikipedia,': [[1], 11], 'free': [[1], 32], 'encyclopedia': [[1], 32], 'wikipedia': [[1], 21], 'article': [[1], 96], 'includes': [[1, 2, 3, 4, 5], 156], ',': [[1, 2, 3, 4, 5], 546], 'related': [[1, 2, 3, 4, 5], 140], 'reading': [[1], 32], 'sources': [[1], 51], 'remain': [[1], 32], 'unclear': [[1], 32], 'lacks': [[1], 32], '.': [[1], 507], '(': [[1], 28], 'october': [[1, 4, 5], 158], '2020': [[1], 96], ')': [[1], 56], 'may': [[1], 32], 'require': [[1], 32], 'specific': [[1], 128], 'problem': [[1], 32], 'is:': [[1], 15], 'n

## 4. Lưu trữ và biểu diễn dữ liệu
Sử dụng pickle lưu lại data với tên file raw_data.
### 4.1 Cơ sở dữ liệu giao tác:
Thông thường, các cơ sở dữ liệu giao tác được lưu trong flat files (các tập phẳng) thay vì trong một hệ cơ sở dữ liệu. Các item là các số nguyên không âm, mỗi giao tác tương ứng với một dòng các số nguyên phân tách nhau bằng khoảng trắng.
Ví dụ:

0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 

30 31 32 

33 34 35 

36 37 38 39 40 41 42 43 44 45 46 

38 39 47 48 

38 39 48 49 50 51 52 53 54 55 56 57 58 

32 41 59 60 61 62 

3 39 48 

63 64 65 66 67 68 

32 69 

48 70 71 72 

39 73 74 75 76 77 78 79 

36 38 39 41 48 79 80 81 

82 83 84 

41 85 86 87 88 

39 48 89 90 91 92 93 94 95 96 97 98 99 100 101 

36 38 39 48 89 

39 41 102 103 104 105 106 107 108 

38 39 41 109 110 

39 111 112 113 114 115 116 117 118 

119 120 121 122 123 124 125 126 127 128 129 130 131 132 133 

48 134 135 136 

39 48 137 138 139 140 141 142 143 144 145 146 147 148 149 

39 150 151 152 

38 39 56 153 154 155 

In [ ]:
for key in data.keys():
  print("\n")
  print(key,":", end=' ')
  for url in data[key][0]:
    print(url, end=" ")
      




web : 1 2 3 4 5 

mining : 1 2 3 

wikipedia, : 1 

free : 1 

encyclopedia : 1 

wikipedia : 1 

article : 1 

includes : 1 2 3 4 5 

, : 1 2 3 4 5 

related : 1 2 3 4 5 

reading : 1 

sources : 1 

remain : 1 

unclear : 1 

lacks : 1 

. : 1 

( : 1 

october : 1 4 5 

2020 : 1 

) : 1 

may : 1 

require : 1 

specific : 1 

problem : 1 

is: : 1 

needs : 1 

sufficient : 1 

inline : 1 

references, : 1 

complete : 1 

re-write : 1 

proposed. : 1 

references : 1 2 3 4 5 

write : 1 

proposed : 1 

name : 1 

proposes, : 1 

information : 1 2 3 

gathered : 1 

web. : 1 

makes : 1 

utilization : 1 

automated : 1 

apparatuses : 1 

reveal : 1 

extricate : 1 

data : 1 

servers : 1 

web2 : 1 

reports, : 1 

permits : 1 

organizations : 1 

get : 1 

organized : 1 2 3 

unstructured : 1 

browser : 1 

activities, : 1 

server : 1 

logs, : 1 

website : 1 

link : 1 

structure, : 1 2 3 

page : 1 

content : 1 

different : 1 

sources.
 : 1 

proposes : 1 

reports

### 4.2 Xuất dataset
Lưu một cơ sở dữ liệu giao tác (transactional database) vào file t_data.txt: 
- Các item tương ứng với url_idx
- Mỗi transaction tương ứng với một từ.

## 5. Truy vấn and, or, not
Vd: 
- Truy vấn `and` câu `web mining`: trả về đường dẫn tới các trang web có cả 2 từ web và từ mining. 
- Truy vấn `or` câu `web mining`: trả về đường dẫn tới các trang web có từ web hoặc từ mining.
- Truy vấn `not` câu `web mining`: trả về đường dẫn tới các trang không có cả từ web và từ mining.

*GỢI Ý: TÁCH CÂU TRUY VẤN THÀNH CÁC TỪ TƯƠNG TỰ PHƯƠNG PHÁP LÀM Ở LAB02.*

In [ ]:
ret={'web': [[1, 2, 3, 4, 5], 2524], 'mining': [[1, 2, 3], 1411], 'wikipedia,': [[1], 11], 'free': [[1], 32], 'encyclopedia': [[1], 32]}

def andRetrieval(ret, string):
    '''Parameters
    -----------------------
    ret: url_list
    string: query'''
    urls={}
    urls_list=[]
    for content in string.split():
      content = content.lower()
      if content in ret.keys():
        if (len(urls)==0):
          urls= set(ret[content][0])
        else:
          temp = set(ret[content][0])
          urls= urls.intersection(temp)
      else:
        return []
    
    for ur in urls:
      urls_list.append(url_list[ur-1])
    return urls_list

def orRetrieval(ret, string):
    '''Parameters
    -----------------------
    ret: url_list
    string: query'''
    urls={}
    urls_list=[]
    for content in string.split():
      content = content.lower()
      if content in ret.keys():
        if (len(urls)==0):
          urls= set(ret[content][0])
        else:
          temp = set(ret[content][0])
          urls= urls.union(temp)
    
    for ur in urls:
      urls_list.append(url_list[ur-1])
    return urls_list

def notRetrieval(ret, string):
    '''Parameters
    -----------------------
    ret: url_list
    string: query'''
    urls={}
    for content in string.split():
      content = content.lower()
      if content in ret.keys():
        if (len(urls)==0):
          urls= set(ret[content][0])
        else:
          temp = set(ret[content][0])
          urls = urls.union(temp)
        ret.pop(content, None)
    
    tmp = list(ret.values())
    urls_not={}
    for i in tmp:
      if (len(urls_not)==0):
        urls_not = set(i[0])
      else:
        temp1 = set(i[0])
        urls_not = urls_not.union(temp1)

    urls_notHaveWordInString=list(urls_not-urls)
 
    urls_list=[]
    for ur in urls_notHaveWordInString:
      urls_list.append(url_list[ur-1])
    return urls_list